<a href="https://colab.research.google.com/github/maryamshahani/BuildingMyNueralNetwork/blob/main/StoryTeller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install PyPDF2

In [21]:
#read-PDF imports here
import PyPDF2

In [22]:
#pre-processing imports here
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
#mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
#file locations on drive
grimm_url = '/content/drive/MyDrive/GoogleColabFiles/StoryTeller/FairytalesByTheBrothersGrimm.txt'
coraline_url = '/content/drive/MyDrive/GoogleColabFiles/StoryTeller/Coraline.pdf'
alice_url = '/content/drive/MyDrive/GoogleColabFiles/StoryTeller/AlicesAdvanturesInWonderland.txt'

In [25]:
#load punctuation symbols
punct = string.punctuation

# **Pre-processing Coraline**

In [26]:
#a function to pre process Coraline by Neil Gaiman

def preprocess_coraline(book):
  '''
  param book: url od a PDF book file
  '''
  output = ""
  data = open(book, 'rb')
  data = PyPDF2.PdfFileReader(book)
  npages = data.getNumPages()
  for i in range(npages):
    page_i = data.getPage(i).extractText()
    output += page_i
  output = output[1227:]
  output = output.lower()
  for word in output:
    for char in word:
        if char in punct:
            word = word.replace(char, "")
  remove_punct = "".join([word for word in output if word not in punct])
  processed = word_tokenize(remove_punct)
  print('Coraline database includes {} tokens, and {} unique tokens after editing'.format(len(processed), len(set(processed)))) 
  return processed

coraline = preprocess_coraline(coraline_url) 

Coraline database includes 30835 tokens, and 3471 unique tokens after editing


## **Preprocessing Alice in Wonderland**

In [27]:
#a function to pre process Alice's Advantures in Wonderland by Lewis Carroll

def load_alice(text_file, punct, not_a_word):
    '''
    param text_file: url to Project Gutenberg's text file for Alice's Advantures in Wonderland by Lewis Carroll
    param punct: a string of punctuation characters we'd like to filter
    param not_a_word: a list of words we'd like to filter
    '''
    book = open(text_file, 'r')
    book = book.read()
    book = book[715:145060]
    book_edit = re.sub('[+]', '', book)
    book_edit = re.sub(r'(CHAPTER \w+.\s)', '', book)
    words = word_tokenize(book_edit.lower())
    
    word_list = []
    
    # filtering punctuation and non-words
    for word in words:
        for char in word:
            if char in punct:
                word = word.replace(char, "")
        if word not in punct and word not in not_a_word:
            word_list.append(word)

    print('Alice database includes {} tokens, and {} unique tokens after editing'.format(len(word_list), len(set(word_list)))) 
    return word_list

alice = load_alice(alice_url, (punct.replace('-', "") + '’' + '‘'), ['s', '--', 'nt', 've', 'll', 'd'])

Alice database includes 26612 tokens, and 2596 unique tokens after editing


# **Preprocessing Grimm**


In [28]:
def load_fairytales(text_file):
    '''
    param text_file: url to Project Gutenberg's text file for Fairytales by The Brothers Grimm
    '''
    book = open(text_file, encoding='cp1252')
    book = book.read()
    book = book[2376:519859]
    book_edit = re.sub('[(+*)]', '', book)
    words = word_tokenize(book_edit.lower())

    # filtering punctuation inside tokens (example: didn't or wow!)
    for word in words:
        for char in word:
            if char in punct:
                word = word.replace(char, "")

    # filtering punctuation as alone standing tokens(example: \ or ,)
    words = [word for word in words if word not in punct]

    print('Fairytales database includes {} tokens, and {} unique tokens after editing'.format(len(words), len(set(words))))            
    return words

brothers_grimm = load_fairytales(grimm_url)

Fairytales database includes 106324 tokens, and 5335 unique tokens after editing


# **Combined database including all books**

In [29]:
data = coraline + alice + brothers_grimm
data[:10]

['coraline',
 'discovered',
 'the',
 'door',
 'a',
 'little',
 'while',
 'after',
 'they',
 'moved']

# **Convert Data into Numeric Values cause Neural Network speaks Math rather than English**

In [30]:
vocab_size = len(data)
print(vocab_size)

vocab = set(data)
unique_vocab = len(vocab)
print(unique_vocab)


163771
7923


In [31]:
word_to_index = { word: i for i, word in enumerate(vocab)}
print(word_to_index)

{'i': 0, 'embraced': 1, 'unrolled': 2, 'incaseofemergency': 3, 'provision': 4, 'diligent': 5, 'warrior': 6, 'glimmer': 7, 'jelly': 8, 'insists': 9, 'yelled': 10, 'lay': 11, 'would': 12, 'oompah': 13, 'brushing': 14, 'snap': 15, 'sat': 16, 'still': 17, 'manner': 18, 'chose': 19, 'neck': 20, 'unconcernedness': 21, 'brighter': 22, 'said': 23, 'chase': 24, 'unharmed': 25, 'faint': 26, 'speak': 27, 'wasteland': 28, 'affair': 29, 'royal': 30, 'countryman': 31, 'swords': 32, 'rising.': 33, 'spoil': 34, 'amends': 35, 'carefully': 36, 'morals': 37, 'delighted': 38, 'shudder—if': 39, 'thieving': 40, 'dash': 41, 'certain': 42, 'yourself': 43, 'hear': 44, 'travel': 45, 'games': 46, 'desire': 47, 'soldiers': 48, 'ended': 49, 'well-deserved': 50, 'huntsman.': 51, 'stained': 52, 'considered': 53, 'troubled': 54, 'nearer': 55, 'off.': 56, 'rightly': 57, 'nondescript': 58, 'seriously': 59, 'frightened': 60, 'follows': 61, 'shinest': 62, 'fits.': 63, 'reassured': 64, 'smooth': 65, 'lifted': 66, 'metal':

In [32]:
data = [word_to_index[word] for word in data]
print(data[:10])

[2222, 4084, 4861, 7886, 811, 639, 915, 2955, 6770, 6789]


In [33]:
#to test the match
print(word_to_index['coraline']) 
print(word_to_index['discovered'])

2222
4084


# **Batching Data (splitting data into Batches)**

In [34]:
batch_size = 5
#storing our features and targets inside the tuple
train_data = [([data[i], data[i+1], data[i+2], data[i+3], data[i+4]], data[i+5]) for i in range (vocab_size - batch_size)]
train_data[:10]

[([2222, 4084, 4861, 7886, 811], 639),
 ([4084, 4861, 7886, 811, 639], 915),
 ([4861, 7886, 811, 639, 915], 2955),
 ([7886, 811, 639, 915, 2955], 6770),
 ([811, 639, 915, 2955, 6770], 6789),
 ([639, 915, 2955, 6770, 6789], 5613),
 ([915, 2955, 6770, 6789, 5613], 4861),
 ([2955, 6770, 6789, 5613, 4861], 5715),
 ([6770, 6789, 5613, 4861, 5715], 5702),
 ([6789, 5613, 4861, 5715, 5702], 5783)]

# **Defining the Neural Network**

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

embedding_dim = 5

class StoryTeller(nn.Module):
  def __init__(self, vocab_size, embedding_dim, batch_size):
    super(StoryTeller,self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(batch_size * embedding_dim, 128) #first linear transformation
    self.linear2 = nn.Linear(128, 512) #we specified the output of previous layer in it
    self.linear3 = nn.Linear(512, vocab_size) #third linear transformation

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1,-1))
    out = F.relu(self.linear1(embeds))
    out = F.relu(self.linear2(out))
    out = self.linear3(out)
    log_probs = F.log_softmax(out , dim=1) #log softmax method
    return log_probs

#test the model so far
model =  StoryTeller(vocab_size, embedding_dim, batch_size)  
model


StoryTeller(
  (embeddings): Embedding(163771, 5)
  (linear1): Linear(in_features=25, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=163771, bias=True)
)

# **Defining Training Function**

# **Train Model**

# **Save Checkpoint**

In [ ]:
checkpoint_url = '/content/drive/My Drive/Lessons/storyTeller/checkpoint3.pth'

checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'word_to_index': word_to_index,
              'index_to_word': {i: word for i, word in enumerate(vocab)},
              'epochs': epochs,
              'average_loss': average_loss,
              'device': device,
              'optimizer_state': optimizer.state_dict(),
              'batch_size': batch_size}

torch.save(checkpoint, checkpoint_url)

# **Load Checkpoint**

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.optimizer_state = checkpoint['optimizer_state']
    model.load_state_dict(checkpoint['state_dict'])
    model.device = checkpoint['device']
    model.word_to_index = checkpoint['word_to_idxx']
    model.index_to_word = checkpoint['idx_to_word']
    model.average_loss = checkpoint['average_loss']
    return model

checkpoint_url = '/content/drive/My Drive/Lessons/storyTeller/checkpoint5.pth'
model = load_checkpoint(checkpoint_url)
index_to_word = model.index_to_word
model

In [ ]:
import pandas as pd

loss_plot = pd.DataFrame(model.average_loss)
loss_plot.plot()

# **Predict Function**

In [ ]:
def predict(model, first_words ,story_len ,top_k):
    '''
    param model: trained model
    param first_words: a string of 5 (n_feature) words to begin the story
    param story_len: an integer symbolizing the number of words you'd like the story to have
    param top_k: the number of top probabilities per word that the network will randomly select from
    '''
    feature = (first_words.lower()).split(" ")
    for i in feature:
        story.append(i)
    for i in range(story_len):
        feature_idx = torch.tensor([word_to_index[word] for word in feature], dtype=torch.long)
        feature_idx = feature_idx.to(device)
        with torch.no_grad():
            output = model.double().forward(feature_idx)
        ps = torch.exp(output)
        topk_combined = ps.topk(top_k, sorted=True)
        #top kk probabilities
        topk_ps = topk_combined[0][0]
        #top kk classes
        topk_class = topk_combined[1][0]
        topk_class = [index_to_word[int(i)] for i in topk_class]
        next_word = random.choice(topk_class)
        feature = feature[1:]
        feature.append(next_word)
        story.append(next_word)
    return story

# **Predict**

In [ ]:
import random
first_words = input('Type the first {} words to start the story:\nexample: A lovely day at the\n'.format(batch_size))

top_k = 3
story_len = 50
story = []
device = 'cuda:0'

#Predicting and Handling User-Input Errors
try:      
    prediction = predict(model, first_words, story_len, top_k)
except KeyError as error:
    print('Oops, looks like you\'ve selected a word that the network does not understand yet: ', error)
    if story[0] != "":
        story = story[len(first_words):]
    first_words = input('please select a different word:\nexample: A lovely day at the\n')
    prediction = predict(model, first_words, story_len, top_k)
except KeyError and RuntimeError:
    if story[0] != "":
        story = story[len(first_words):]
    first_words = input('Oops, looks like you\'ve typed {} words instead of {}!\n\nType the first 5 words to start the story:\nexample: A lovely day at the\n'.format(len(first_words.split(" ")), n_features))
    prediction = predict(model, first_words, story_len, top_k)

print('-----------------------------------------------------\n The STORY \n-----------------------------------------------------')
print(" ".join(story))